In [1]:
import geopandas as gpd
import fiona
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import configparser
import momepy
import libpysal
import osmnx
import pandas
from bokeh.io import output_notebook
from bokeh.plotting import show
from clustergram import Clustergram
from shapely.geometry import Point
local_crs = "EPSG:2039"
output_notebook()

Loading BokehJS ...

Load all the gdb files from michael folders

In [2]:

config = configparser.ConfigParser()
config_file = 'config.ini'
# check if file exists
if not os.path.isfile(config_file):
    # create a simple config file
    config['Paths'] = {'root_folder': ''}
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

config.read('config.ini')

root_folder = config['Paths']['root_folder']  
print(root_folder)
root_folder = os.path.join(root_folder, "Michaels_Data")
print(root_folder)
# List all .gdb directories in the folder
gdb_files = []
for dirpath, dirnames, filenames in os.walk(root_folder):
    # Check if the folder contains a "commondata" folder
    if "commondata" in dirnames:
        commondata_folder = os.path.join(dirpath, "commondata")
        commondata_folder = os.path.normpath(commondata_folder)

        # List all .gdb files in the "commondata" folder
        for f in os.listdir(commondata_folder):
            if f.endswith('.gdb'):
                gdb_files.append(os.path.join(commondata_folder, f))

# Print the collected .gdb files
for gdb in gdb_files:
    print(gdb)
print("GDB Files:", gdb_files)


Michaels_Data
Michaels_Data\חלקות\commondata\myproject16.gdb
Michaels_Data\חלקות\commondata\scratch.gdb
Michaels_Data\מרקמים\commondata\myproject16.gdb
Michaels_Data\מרקמים\commondata\scratch.gdb
Michaels_Data\קונטור בניינים\commondata\jps_reka.gdb
Michaels_Data\קונטור בניינים\commondata\scratch.gdb
Michaels_Data\רגישות\commondata\myproject16.gdb
GDB Files: ['Michaels_Data\\חלקות\\commondata\\myproject16.gdb', 'Michaels_Data\\חלקות\\commondata\\scratch.gdb', 'Michaels_Data\\מרקמים\\commondata\\myproject16.gdb', 'Michaels_Data\\מרקמים\\commondata\\scratch.gdb', 'Michaels_Data\\קונטור בניינים\\commondata\\jps_reka.gdb', 'Michaels_Data\\קונטור בניינים\\commondata\\scratch.gdb', 'Michaels_Data\\רגישות\\commondata\\myproject16.gdb']


Calculating metrics over buildings:

In [3]:
# load municipal buildings
gdb_file = gdb_files[4]  # Modify if you want to choose a different .gdb
print(gdb_file)
# List the layers in the selected .gdb
layers = fiona.listlayers(gdb_file)
print("Layers in the selected GDB:", layers)

# Choose a specific layer within the .gdb
textures_layer = layers[0]  # Modify if you want to choose a different layer

# Load the specific layer
gdf = gpd.read_file(gdb_file, layer=textures_layer)
buildings = gdf[["geometry"]].to_crs(local_crs)


Michaels_Data\קונטור בניינים\commondata\jps_reka.gdb
Layers in the selected GDB: ['buildings']


In [ ]:

# Calculate Building Height (assuming you have the necessary columns like number of floors)
# You need to have a 'height_col' or a similar column to calculate this
# buildings['building_height'] = momepy.building_height(buildings, 'height_col').series

# Calculate Volume, requires floor_area and height
# Assuming height and area have been calculated and are stored in 'height' and 'floor_area' columns
# buildings['volume'] = momepy.Volume(buildings, 'floor_area', 'building_height')
# buildings['form_factor'] = momepy.form_factor(buildings, 'building_height')
# buildings['floor_area'] = momepy.floor_area(buildings,'building_height')

# Basic geometric properties
buildings['perimeter'] = buildings.geometry.length
buildings['shape_index'] = momepy.shape_index(buildings,momepy.longest_axis_length(buildings))
buildings['circular_compactness'] = momepy.circular_compactness(buildings)
buildings['square_compactness'] = momepy.square_compactness(buildings)
buildings['weighted_axis_compactness'] = momepy.compactness_weighted_axis(buildings)
buildings['convexity'] = momepy.convexity(buildings)
buildings['courtyard_area'] = momepy.courtyard_area(buildings)
buildings['courtyard_index'] = momepy.courtyard_index(buildings)
# buildings['corners'] = momepy.corners(buildings)                # TODO: Fix this
buildings['fractal_dimension'] = momepy.fractal_dimension(buildings)
buildings['facade_ratio'] = momepy.facade_ratio(buildings)


In [25]:
# More complex morphological metrics
buildings['orientation'] = momepy.orientation(buildings)
buildings['longest_axis_length'] = momepy.longest_axis_length(buildings)
buildings['equivalent_rectangular_index'] = momepy.equivalent_rectangular_index(buildings)
buildings['elongation'] = momepy.elongation(buildings)
buildings['linearity'] = momepy.linearity(buildings)
buildings['rectangularity'] = momepy.rectangularity(buildings)
# buildings['squareness'] = momepy.squareness(buildings)    # TODO: Fix this
buildings['shared_walls_length'] = momepy.shared_walls(buildings)


In [11]:
# Metrics related to building adjacency and Graph
from libpysal import graph

delaunay = graph.Graph.build_triangulation(buildings.centroid, coplanar='clique').assign_self_weight()
orientation = momepy.orientation(buildings)
buildings['alignment'] = momepy.alignment(orientation, delaunay)

buildings_knn = graph.Graph.build_knn(buildings.centroid, k=15)  # adjust k if needed   TODO: Fix this
contiguity = graph.Graph.build_contiguity(buildings)
buildings['adjacency'] = momepy.building_adjacency(contiguity,buildings_knn)

buildings['mean_interbuilding_distance'] = momepy.mean_interbuilding_distance(buildings, delaunay, buildings_knn)
# 
buildings['neighbour_distance'] = momepy.neighbor_distance(buildings, delaunay)


buildings['courtyards_num'] = momepy.courtyards(buildings, contiguity) # Calculate the number of courtyards within the joined structure

# the following metrics caculate the diversity index of building types, it possible to override the binning method (refer to the full documentation)
knn5 = graph.Graph.build_knn(buildings.centroid, k=5)
buildings['shannon'] = momepy.shannon(buildings, knn5) 
buildings['simpson'] = momepy.simpson(buildings, knn5) 
buildings['theil'] = momepy.theil(buildings, knn5)

buildings['values_range'] = momepy.values_range(buildings, knn5) 
buildings['mean_deviation'] = momepy.mean_deviation(buildings, knn5)

CoplanarError: There are 34330 unique locations in the dataset, but 34339 observations. At least one of these sites has 1 points, more than the 15 nearest neighbors requested. This means there are more than 15 points in the same location, which makes this graph type undefined. To address this issue, consider setting `coplanar='clique'` or consult the documentation about coplanar points.

In [ ]:
# metrics related to tessellation
# buildings = momepy.preprocess(buildings.reset_index(), size=30, compactness=0.2, islands=True)
limit = momepy.buffered_limit(buildings, buffer = 'adaptive',coplanar='clique')
# tessellation = momepy.morphological_tessellation(buildings) # need adjustment
# blg_orient = momepy.orientation(buildings)
# tess_orient = momepy.orientation(tessellation)
# buildings['cell_orientation'] = momepy.cell_alignment(blg_orient, tess_orient).series
# buildings['num_of_neighbours'] = momepy.neighbors(tessellation, contiguity, weighted=True)

In [ ]:
# shared metrics between buildings and streets
# Ensure you have the necessary context (like street networks) for these:
# gdf_network =   # the street network
# buildings['node_density'] = momepy.node_density(gdf_network, buildings, radius=300)
# buildings['street_profile'] = momepy.street_profile(gdf_network, buildings, radius=300)
# buildings['reach'] = momepy.Reach(buildings, gdf_network, distance=500)

blg_orient = momepy.orientation(buildings)
str_orient = momepy.orientation(streets)
momepy.street_alignment(blg_orient, str_orient, buildings["street_index"])


Calculate metrics over streets

In [ ]:
place = "Jerusalem, Israel"
local_crs = "EPSG:2039"

# Geocode using Nominatim
# latitude = 31.7683
# longitude = 35.2137
# point = gpd.GeoDataFrame(geometry=[Point(longitude, latitude)], crs="EPSG:4326")

#Load open street buildings
osm_buildings = osmnx.features_from_place(place, tags={"building": True})
osm_buildings = osm_buildings[osm_buildings.geom_type == "Polygon"].reset_index(drop=True)
osm_buildings = osm_buildings[["geometry"]].to_crs(local_crs)
osm_buildings.head()
osm_graph = osmnx.graph_from_place(place, network_type="drive")
osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)
streets = osmnx.graph_to_gdfs(
    osmnx.convert.to_undirected(osm_graph),
    nodes=False,
    edges=True,
    node_geometry=False,
    fill_edge_geometry=True,
).reset_index(drop=True)
print(streets.columns)
streets_geometry = streets['geometry']
streets['orientation'] = momepy.orientation(streets_geometry)
streets['courtyard_area'] = momepy.courtyard_area(streets_geometry)
streets['longest_axis_length'] = momepy.longest_axis_length(streets_geometry)
# streets['perimeter_wall'] = momepy.perimeter_wall(streets_geometry)
streets['centroid_corner_distance'] = momepy.centroid_corner_distance(streets_geometry)
streets['circular_compactness'] = momepy.circular_compactness(streets_geometry)
streets['compactness_weighted_axis'] = momepy.compactness_weighted_axis(streets_geometry)
streets['convexity'] = momepy.convexity(streets_geometry)
streets['corners'] = momepy.corners(streets_geometry)
streets['courtyard_index'] = momepy.courtyard_index(streets_geometry)
streets['elongation'] = momepy.elongation(streets_geometry)
streets['equivalent_rectangular_index'] = momepy.equivalent_rectangular_index(streets_geometry)
streets['facade_ratio'] = momepy.facade_ratio(streets_geometry)
streets['fractal_dimension'] = momepy.fractal_dimension(streets_geometry)
streets['linearity'] = momepy.linearity(streets_geometry)
streets['rectangularity'] = momepy.rectangularity(streets_geometry)
streets['shape_index'] = momepy.shape_index(streets_geometry)
streets['square_compactness'] = momepy.square_compactness(streets_geometry)
streets['squareness'] = momepy.elongation(streets_geometry)
streets['alignment'] = momepy.alignment(streets['orientation'], osm_graph)
streets['cell_alignment'] = momepy.cell_alignment(streets['orientation'], osm_graph)
streets['neighbor_distance'] = momepy.neighbor_distance(streets_geometry, osm_graph)
streets['neighbors'] = momepy.neighbors(streets_geometry, osm_graph)
streets['betweenness_centrality'] = momepy.betweenness_centrality(osm_graph)



streets = momepy.remove_false_nodes(streets)
streets["length"] = streets.length
streets["linearity"] = momepy.linearity(streets)